# Download large amount of stock data from the past 25 years. 

The intention for this notebook is to download large amount of daily stock data (adjusted close prices) via Yahoo finance from 1997-01-01 till 2022-04-01. The data is used for large-scale empirical investigations. 

The ticker data was downloaded from https://dumbstockapi.com/. 

## Libraries:

In [169]:
import yfinance as yf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load stock data:

Tickers are first loaded into the notebook. 

In [170]:
tickers = pd.read_csv('Tickers.csv', delimiter = ',')
tickers = tickers['ticker']
tickers.head()

0       A
1      AA
2    AABA
3     AAC
4     AAL
Name: ticker, dtype: object

In [ ]:
data = []
volume = []
for i in tickers: 
    stockdata = yf.download(i, start = "1997-01-01", end = "2022-04-01")[['Adj Close', 'Volume']]
    data.append(stockdata['Adj Close'])
    volume.append(stockdata['Volume'])


PandasData = pd.DataFrame(data)
PandasData = PandasData.transpose()
PandasData.columns = tickers

PandasVolume = pd.DataFrame(volume)
PandasVolume = PandasVolume.transpose()
PandasVolume.columns = tickers

## Cleaning delisted stocks and unknown tickers from dataframe:

As per this [SEC study](https://www.sec.gov/rules/policy/2019/thinly-traded-securities-tm-background-paper.pdf) we define stocks as illiquid, if they have an average daily trading volume less than 100000 shares. Illiquid stocks tends to be much more sensitive to small exogenous events and thus they have the ability to "skew" results that are otherwise in line with the general market behaviour.  

In [187]:
#Removing all nan columns and rows: 
PandasData = PandasData.dropna(axis=1, how='all')
PandasData = PandasData.dropna(axis=0, how='all')

PandasVolume = PandasVolume.dropna(axis=1, how='all')
PandasVolume = PandasVolume.dropna(axis=0, how='all')


In [198]:
#there are negative values. We remove columns with negative values and average daily trading volume less than 100000 stocks:
PandasData = PandasData.drop(PandasData.columns[(PandasVolume.mean() < 100000)], axis = 1)
PandasData = PandasData.drop(PandasData.columns[(PandasData < 0).any()], axis = 1)

#sees difference in columns:
#set(PandasVolume.columns) ^ set(PandasData.columns)


## Plotting first 20 stocks in dataframe

In [ ]:
sns.set()
sns.set_style("white")
plt.figure(figsize = (16,8))
plt.margins(x=0)
plt.plot(PandasData.index, PandasData.iloc[:,0:20])

plt.show()


## Exporting to Excel CSV file

In [203]:
PandasData.to_csv('AdjCloseLargeStockDB.csv', encoding='utf-8')